# Mounting Drive 


In [1]:
import pandas as pd

# if Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains '.csv'"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))


link = "https://drive.google.com/file/d/1ZpS3-3KqUHV1A5-vdwGDoYaN5RlUwWEc/view?usp=sharing"
url = listed[0]['webContentLink']
df1 = pd.read_csv(url)



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

title movies_SORTED.csv, id 1ZpS3-3KqUHV1A5-vdwGDoYaN5RlUwWEc


In [ ]:
import os
import warnings
import pandas as pd
import statistics
import numpy as np
import seaborn as sns


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import TransformedTargetRegressor

from sklearn.model_selection import train_test_split
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

import matplotlib
from matplotlib        import pyplot as plt
from matplotlib.pyplot import figure


plt.style.use('ggplot')












def mean(x):
  return sum(x)/len(x)

def exclui_outliers(df1, col_name):
  Q1 = df1[col_name].quantile(.25)
  Q3 = df1[col_name].quantile(.75)
  IIQ =Q3 -Q1
  limite_inf = Q1 -1.5*IIQ
  limite_sup = Q3 +1.5*IIQ
  return df1[(df1[col_name]>=limite_inf) & (df1[col_name]<=limite_sup)]

def removeNaN(df, col_name):
  for index, row in enumerate(df[col_name].to_numpy()):
      if str(row) == "nan":
        # print(row)
      # if np.isnan(row): 
        df[col_name][index] = str(float(0))
  return df

def group_low_freq_cats(df1, col_name, threshold=0.01, name='others'):
  df1 = df1.copy()
  cat_freq = df1[col_name].value_counts()
  cat_low_freq = cat_freq[cat_freq/cat_freq.sum() <= threshold].index
  df1.loc[df1[col_name].isin(cat_low_freq),col_name]='others'
  return df1

def val_couts_cols (df1,cols):
  """ Problems Here"""

  for x in cols:
    print('column: {0}, categories: {1}'.format(x,len(df1[x].value_counts())))
  print('Total samples: ' + str(len(df1)))

  def feature_selection(df1, feature, target ,in_out, method='na'):fs_score =[]
  oe = OrdinalEncoder()
  X = (np.array(df1[feature])).reshape(-1,1)
  oe.fit(X)
  X_enc = oe.transform(X)

  y = np.array(df1[target]).reshape(-1,1)
  oe.fit(y)
  y_enc = oe.transform(y)
  
  if in_out == 'cat_cat': 
    if method == 'chi2':
      fs = SelectKBest(score_func=chi2, k='all') 
    else:
      fs = SelectKBest(score_func=mutual_info_classif, k='all')
    fs.fit(X_enc, y_enc)
    fs_score = fs.scores_
  elif in_out == 'num_num':
    fs = SelectKBest(score_func=f_regression, k='all')
    fs.fit(X, y.ravel())
    fs_score = fs.scores_
  elif in_out == 'num_cat':
    fs = SelectKBest(score_func=f_classif, k='all')
    fs.fit(X, y_enc)
    fs_score = fs.scores_
  elif in_out == 'cat_num':
    fs = SelectKBest(score_func=f_classif, k='all')
    fs.fit(X_enc, y.ravel())
    fs_score = fs.scores_
  else:
    fs_score=[]
  return fs_score



def get_col_type(df1,col_type):
  cols_types=df1.dtypes.reset_index()
  cols_types.columns=['col','type']
  cols_type = cols_types.apply(lambda x: x['col'] if x['type']==col_type else np.nan ,axis=1)
  return cols_type.dropna()

def boxplot_by_col(df1,cat_cols,target):
  fig, ax = plt.subplots(len(cat_cols), 1, figsize=(25, 18))
  fig.subplots_adjust()
  t=0
  for var, subplot in zip(cat_cols, ax.flatten()):
      ax[t].set_xlabel(var,fontsize=18)
      sort_qtl_index = df1.groupby(var)[target].quantile(0.5).sort_values().index
      sort_qtl_values = df1.groupby(var)[target].quantile(0.5).sort_values()
      sns.boxplot(x=var, y=target, data=df1, ax=subplot,order=sort_qtl_index)
      sns.pointplot(x=sort_qtl_index,y= sort_qtl_values,ax=subplot,color='r')
      t+=1    
  plt.tight_layout(pad=3) 



def group_low_freq_cats(df1, col_name, threshold=0.01, name='others'):
  df1 = df1.copy()
  cat_freq = df1[col_name].value_counts()
  cat_low_freq = cat_freq[cat_freq/cat_freq.sum() <= threshold].index
  df1.loc[df1[col_name].isin(cat_low_freq),col_name]='others'
  return df1


def remove_incoherence(DataFrame,expression, replace_val, columns=[]):
  """ Problems Here"""
  
  if len(columns)==0:
    columns = DataFrame.columns
  
  DataFrame_aux=DataFrame.copy()
  
  if str(replace_val) == str(np.nan):
    DataFrame_aux=DataFrame.replace(expression, replace_val, regex=True) 
    return DataFrame_aux
  else: 
    for col in columns:
      i=0
      while True: 
        DataFrame_aux[col]=DataFrame[col].str.replace(expression, replace_val, regex=True)
        num_matchs = len(DataFrame_aux[DataFrame_aux[col].str.contains(expression, na=False)])
        DataFrame = DataFrame_aux
        if num_matchs == 0:
            break
        if i == 100:
            DataFrame_aux =pd.DataFrame([])
            break
        i+=1
    return DataFrame_aux

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors, NeighborhoodComponentsAnalysis, KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

def trainCluster(X,y):
    """KNeighborsClassifier"""
    model = KNeighborsClassifier(n_neighbors=3)
    print (len(X), y[0] )
    print (len(X), y[0], set(y))
    return model.fit(X, y)


def modelEval(model, XTests, yTests):
    yPreds = model.predict(XTests)
    for idx,_ in enumerate(yPreds):
      yTest = yTests[idx]
      yPred = yPreds[idx]
      print(idx, yPred, "---",yTest)
    results = cross_val_score(model, X, y, cv=5)
    print(results)
    return 





def test():
  X, y = load_iris(return_X_y=True)
  print (X.shape)
  print (y.shape)
  
  # X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.7, random_state=42)
  # knn = trainCluster(X,y)
  # modelEval(knn, X_test, y_test)

  # def runClustering():
  #     """ """
  #     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
  #     knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
  #     knn.fit(X_train, y_train)
  #     y_pred = knn.predict(X_test)
  #     sns.scatterplot(
  #         x='mean gross',
  #         y='mean budget',
  #         hue='4500000',
  #         data=X_test.join(y_test, how='outer'))
  #     cat_cols = get_col_type(df1_fs, 'object')
  #     # val_couts_cols(df1_fs,cat_cols)



(150, 4)
(150,)


In [ ]:
link = "https://drive.google.com/file/d/1ZpS3-3KqUHV1A5-vdwGDoYaN5RlUwWEc/view?usp=sharing"
url = listed[0]['webContentLink']
df1 = pd.read_csv(url)





def clean_data(data, n = 5):
  y_col = ["gross", "budget"]
  X_col = ["rating", "score"] # ["rating", "genre", "score", "company", ""]
  df = removeNaN(data, "rating")
  df = removeNaN(df, "score")
  df = removeNaN(df, "gross")
  df = removeNaN(df, "budget")

  df_ratings = [str(i) for i in df.rating.values]
  ratingsLT = list(set(df_ratings))

  print(f"RatingsLT: {ratingsLT}")
  for idx, row in enumerate(df.rating):
    df.rating[idx] = ratingsLT.index(row)
  return df





if __name__ == "__main__":
    df_clean = clean_data(data = df1)
    y = df_clean[y_col[0]].to_numpy()
    X = df_clean[X_col].to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)
    knn = trainCluster(X,y)
    modelEval(knn, X_test, y_test)
    
    

  # X =s 

  # df1_fs = df1.drop(['country','votes','runtime','released'], axis=1)
  # print (f"df1: {df1_fs}")
  # print (f"df1 info: {df1_fs.info}")
  # df1_fs = removeNaN(df1_fs, "budget")
  # df1_fs = removeNaN(df1_fs, "gross")
  # print ()
  # print (df1_fs['gross'])

  # print (df1_fs['budget'].plot(kind='box'))
  # print (df1_fs['gross'].plot(kind='box'))

  # gross_budget = df1_fs[["gross","budget"]]
  # gross_mean   = mean(x = df1_fs["gross"])
  # budget_mean  = mean(x = df1_fs["budget"])

  # print(gross_budget)
  # print(gross_mean)
  # print(budget_mean)
  
  # ============================================================ #
  # df1.sample(500)
  # df1.info()
  # df1_fs.info()
  # df1_fs.sample(15)
  # df1.dropna()
  # ============================================================ #
  # sns.heatmap(df1_fs.corr(), annot=Tdrue)
  # sns.pairplot(df1_fs[['budget','gross','genre']])

  # for dirname, _, filenames in os.walk('/kaggle/input'):
  #     for filename in filenames:
  #         print(os.path.join(dirname, filename))
  # warnings.filterwarnings('ignore')
  # %matplotlib inline
  # matplotlib.rcParams['figure.figsize'] = (12,8)
  # pd.options.mode.chained_assignment = None
  # df1 = pd.read_csv('https://drive.google.com/uc?id=1ZpS3-3KqUHV1A5-vdwGDoYaN5RlUwWEc&export=download')
  # df1.head()


  # # print (y)
  # # print(df1.target)


In [ ]:
# X = X[[mean_gross, mean_budget]]
# y = pd.Categorical.from_codes(df1_gross.target, df1_gross.target_names)
# y = pd.get_dummies(y, drop_first=True)

print(df1.isnull().sum())
print(df1["gross"])

df1.dropna(inplace = True)
print(df1.isnull().sum())
df1['gross'].astype('int64')
df1['budget'].astype('int64')


name        0
rating      0
genre       0
year        0
released    0
score       0
votes       0
director    0
writer      0
star        0
country     0
budget      0
gross       0
company     0
runtime     0
dtype: int64
0       758411779.0
1       433921300.0
2       400063852.0
3       359126022.0
4       335260290.0
           ...     
1345    127869379.0
1346    161849455.0
1349     54323210.0
1350     36599361.0
1351     52425855.0
Name: gross, Length: 1174, dtype: float64
name        0
rating      0
genre       0
year        0
released    0
score       0
votes       0
director    0
writer      0
star        0
country     0
budget      0
gross       0
company     0
runtime     0
dtype: int64


0       180000000
1       150000000
2       170000000
3        40000000
4        79000000
          ...    
1345     50200000
1346     27000000
1349     13200000
1350      8000000
1351      4000000
Name: budget, Length: 1174, dtype: int64

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey = True)

plt.gcf().set_size_inches(15, 7)

ax1.boxplot(df1.budget)

ax1.set_title('Budget', c = 'green', fontsize = 25)

ax2.boxplot(df1.gross)

ax2.set_title('Gross', c ='red', fontsize = 25)

plt.ylabel('Gross', fontsize = 25)

plt.show()

In [ ]:
correlation_table = df1.corr(method = 'pearson')

In [ ]:
sns.heatmap(correlation_table, annot = True)

plt.title('Correlation Heat Map')

plt.xlabel('Measuring Features')

plt.ylabel('Measuring Features')

plt.show()


In [ ]:
coded_df1 = df1

for col_name in coded_df1.columns:
   
    if(coded_df1[col_name].dtype == 'object'):
      
        coded_df1[col_name] = coded_df1[col_name].astype('category')
       
        coded_df1[col_name] = coded_df1[col_name].cat.codes

coded_df1.head()

In [ ]:
complete_corr_table = coded_df1.corr(method = 'pearson')

sns.heatmap(complete_corr_table, annot = True)

plt.title('Complete Correlation Heat Map')

plt.xlabel('Measuring Features')

plt.ylabel('Measuring Features')

plt.show()

In [ ]:
pairs = complete_corr_table.unstack()
sorted_pairs = pairs.sort_values()
high_correlation = sorted_pairs[(sorted_pairs) > 0.5]



print (pairs)
print (sorted_pairs)
print (high_correlation)

sns.regplot(x='score', y='gross', data=df1, scatter_kws={'color': 'magenta'}, line_kws = {'color': 'blue'})
sns.regplot(x='votes', y='gross', data=df1, scatter_kws={'color': 'green'}, line_kws = {'color': 'blue'})
sns.regplot(x='budget', y='gross', data=df1, scatter_kws={'color': 'red'}, line_kws = {'color': 'yellow'})
sns.stripplot(x="rating", y="gross", data = df1)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import PowerTransformer
from sklearn.compose import TransformedTargetRegressor

from sklearn.model_selection import train_test_split

from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn import metrics

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

dataset = pd.read_csv('https://drive.google.com/uc?id=1ZpS3-3KqUHV1A5-vdwGDoYaN5RlUwWEc&export=download',encoding='latin1')
dataset.sample(100)




In [ ]:
def exclui_outliers(DataFrame, col_name):

  Q1 = DataFrame[col_name].quantile(0.25)

  Q3 = DataFrame[col_name].quantile(0.75)

  IIQ =Q3 -Q1

  limite_inf = Q1 -1.5*IIQ

  limite_sup = Q3 +1.5*IIQ


  return DataFrame[(DataFrame[col_name]>=limite_inf & (DataFrame[col_name]<=limite_sup)]
                    
def group_low_freq_cats(DataFrame, col_name, threshold=0.01, name='others'):
  
  df1 = DataFrame.copy()
  
  cat_freq = df1[col_name].value_counts()
  
  cat_low_freq = cat_freq[cat_freq/cat_freq.sum() <= threshold].index
  
  df1.loc[df[col_name].isin(cat_low_freq),col_name]='others'
  
  return df1


def val_couts_cols (Dataframe,cols):
  for x in cols:
    print('column: {0}, categories: {1}'.format(x,len(Dataframe[x].value_counts())))
  print('Total samples: ' + str(len(Dataframe)))

#dataset_fs = exclui_outliers(dataset_fs, 'budget', 'gross')
#dataset_fs['budget'].plot(kind='box') & dataset_fs['gross'].plot(kind='box')


dataset_fs = dataset.drop(['name','country','score', 'votes'],axis=1)
dataset_fs['released'] = pd.to_datetime(dataset_fs['released'])
dataset_fs['released']=(dataset_fs['released'].dt.month).astype('object')
dataset_fs[dataset_fs['gross']==0.0]= np.nan # Problem Here 
dataset_fs=dataset_fs.dropna()
dataset_fs.sample(15)


print (dataset.info())
print (dataset_fs.info())



In [ ]:
plt.figure(figsize=(13,9))
sns.heatmap(dataset_fs.corr(),annot=True)
sns.pairplot(dataset_fs[['gross','runtime','year','budget']])

dataset_fs['budget'].plot(kind='box')
dataset_fs['gross'].plot(kind='box')

In [ ]:
def feature_selection(Dataset, feature, target ,in_out, method='na'): 
  
  fs_score =[]
  
  oe = OrdinalEncoder()

  
  X = (np.array(Dataset[feature])).reshape(-1,1)
  
  oe.fit(X)
  
  X_enc = oe.transform(X)

  
  y = np.array(Dataset[target]).reshape(-1,1)
  
  oe.fit(y)
  
  y_enc = oe.transform(y)
  
  
  if in_out == 'cat_cat': 
  
    if method == 'chi2':
  
      fs = SelectKBest(score_func=chi2, k='all') 
  
    else:
  
      fs = SelectKBest(score_func=mutual_info_classif, k='all')
  
    fs.fit(X_enc, y_enc)
  
    fs_score = fs.scores_
  
  elif in_out == 'num_num':
  
    fs = SelectKBest(score_func=f_regression, k='all')
  
    fs.fit(X, y.ravel())
  
    fs_score = fs.scores_
  
  elif in_out == 'num_cat':
  
    fs = SelectKBest(score_func=f_classif, k='all')
  
    fs.fit(X, y_enc)
  
    fs_score = fs.scores_
  
  elif in_out == 'cat_num':
  
    fs = SelectKBest(score_func=f_classif, k='all')
  
    fs.fit(X_enc, y.ravel())
  
    fs_score = fs.scores_
  
  else:
  
    fs_score=[]

  
  return fs_score

In [ ]:
def get_col_type(df1,col_type):
  
  cols_types=df1.dtypes.reset_index()
  
  cols_types.columns=['col','type']
  
  cols_type = cols_types.apply(lambda x: x['col'] if x['type']==col_type else np.nan ,axis=1)
  
  return cols_type.dropna()

In [ ]:
def boxplot_by_col(df1,cat_cols,target):
  
  fig, ax = plt.subplots(len(cat_cols), 1, figsize=(25, 18))
  
  fig.subplots_adjust()
  
  t=0
  
  for var, subplot in zip(cat_cols, ax.flatten()):
  
      ax[t].set_xlabel(var,fontsize=18)
  
      sort_qtl_index = df.groupby(var)[target].quantile(0.5).sort_values().index
  
      sort_qtl_values = df.groupby(var)[target].quantile(0.5).sort_values()
  
      sns.boxplot(x=var, y=target, data=df, ax=subplot,order=sort_qtl_index)
  
      sns.pointplot(x=sort_qtl_index,y= sort_qtl_values,ax=subplot,color='b')
  
      t+=1    
  
  plt.tight_layout(pad=3) 

In [ ]:
def remove_incoherence(DataFrame,expression, replace_val, columns=[]):
  
  if len(columns)==0:
    
    columns = DataFrame.columns
  
  DataFrame_aux=DataFrame.copy()
  
  if str(replace_val) == str(np.nan):
    
    DataFrame_aux=DataFrame.replace(expression, replace_val, regex=True) # do not use str.replace because it doesn't accept np.nan
    
    return DataFrame_aux
  
  else: 
  
    for col in columns:
      i=0
  
      while (True): # when working with groups in the regex, it is not able to replace all groups, so it is necessary to iterate with each new replacement.
        
        DataFrame_aux[col]=DataFrame[col].str.replace(expression, replace_val, regex=True)

        #warnings.filterwarnings('ignore','UserWarning') # avoid warning when str.contains calls expressions containing groups that will not be used
        
        num_matchs = len(DataFrame_aux[DataFrame_aux[col].str.contains(expression, na=False)])#  check if the regex worked, if so it returns 0, otherwise it returns the number of matches
        
        DataFrame = DateFrame_aux
       
        
        if num_matchs == 0:
           
            break
        
        if i == 100:
        
            DataFrame_aux =pd.DataFrame([])
        
            break
        
        i+=1
    
    return DataFrame_aux

In [ ]:
cat_cols = get_col_type(dataset_fs, 'object')
val_couts_cols(dataset_fs,cat_cols)


In [ ]:
# Sheldon is Here!!!!
import pandas as pd

def clean_dataset(df1):
  assert isinstance(df1, pd.DataFrame), "df1 needs to be a pd.DataFrame"
  df1.dropna(inplace=True)
  indices_to_keep = ~df1.isin([np.nan, np.inf, -np.inf]).any(1)
  return df1[indices_to_keep].astype(np.float64)


fs_scores = [ ]
for x in cat_cols:
    fs_scores - feature_selection(dataset_fs, x, 'budget', 'cat_num')
    print('column: {0}, fs_scores: {1}'.format(x, fs_scores))
    fs_scores.append(fs_scores)
    
np.mean(fs_scores)